# MCP Agents with LLM Integration

**ADVANCED NOTEBOOK** - Run `mcp_client_demo.ipynb` first to understand the basics!

This notebook demonstrates how to use LLMs (Large Language Models) with MCP servers to create intelligent agents that can:
- Understand natural language requests
- Decide which MCP tools to use
- Execute actions through MCP servers
- Provide intelligent responses

## Two Integration Methods:

1. **Ollama (Local Models)**: Run models locally without API keys
2. **API-based Models**: Use OpenAI or Google Gemini with API keys

## Setup

Make sure you have:
- ✅ Completed `mcp_client_demo.ipynb` first
- ✅ Installed dependencies: `pip install -r requirements.txt`
- ✅ For Ollama: Install and run Ollama locally (https://ollama.ai)
- ✅ For API models: Set up API keys in `.env` file (see `API_KEYS_SETUP.md`)

### API Keys Setup (Cloud Models)

**No interactive prompts!** API keys are read from `.env` file:

1. Create `.env` file in project root
2. Add your keys:
   ```
   OPENAI_API_KEY=your-openai-key-here
   GEMINI_API_KEY=your-gemini-key-here
   ```
3. Get keys from:
   - OpenAI: https://platform.openai.com/api-keys
   - Gemini: https://aistudio.google.com/app/apikey

See `API_KEYS_SETUP.md` for detailed instructions.

## How to Use

1. **Run cells ONE AT A TIME**
2. **Wait for each cell to complete** (look for `[1]`, `[2]`, etc.)
3. **`[*]` means running** - wait for it to finish
4. **If stuck**: Press `Ctrl+C` (or `Cmd+C`) to interrupt, then restart kernel

## Note: Execution Numbers and Timestamps

If you don't see cell execution numbers or timestamps:
- **JupyterLab**: Go to View → Show Line Numbers, or Settings → Advanced Settings Editor → Notebook → enable `showExecutionTime`
- **Jupyter Notebook**: Execution numbers should appear automatically when you run cells
- **VS Code**: Check Jupyter extension settings for execution display options


In [1]:
import asyncio
import json
import os
import sys
from pathlib import Path
from typing import Optional

# Load environment variables from .env file FIRST (before other imports that might need them)
from dotenv import load_dotenv

# Try multiple paths for .env file
# 1. Try project root (parent of notebooks/)
env_paths = [
    Path("../.env").resolve(),  # From notebooks/ directory
    Path(".env").resolve(),     # Current working directory
    Path(__file__).parent.parent / ".env" if '__file__' in globals() else None,  # Absolute from file
]

# Also try loading from current directory (like test notebook does)
load_dotenv()  # This loads from current working directory

# Then try explicit paths
for env_path in env_paths:
    if env_path and env_path.exists():
        load_dotenv(env_path, override=False)  # Don't override if already loaded
        print(f"✓ Loaded .env from: {env_path}")
        break
else:
    print("ℹ️  No .env file found - using environment variables only")

# MCP imports
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Get the Python interpreter from the current environment (venv)
# This ensures servers use the same Python that has MCP installed
PYTHON_EXECUTABLE = sys.executable
print(f"Using Python: {PYTHON_EXECUTABLE}")

# Verify API keys are loaded (for debugging)
if os.getenv("OPENAI_API_KEY"):
    key = os.getenv("OPENAI_API_KEY")
    print(f"✓ OPENAI_API_KEY loaded: {key[:10]}...{key[-4:]} (length: {len(key)})")
if os.getenv("GEMINI_API_KEY"):
    key = os.getenv("GEMINI_API_KEY")
    print(f"✓ GEMINI_API_KEY loaded: {key[:10]}...{key[-4:]} (length: {len(key)})")

# LLM imports
try:
    import ollama
    OLLAMA_AVAILABLE = True
except ImportError:
    OLLAMA_AVAILABLE = False
    print("Ollama not installed. Install with: pip install ollama")

try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("OpenAI not installed. Install with: pip install openai")

try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False
    print("Google Generative AI not installed. Install with: pip install google-generativeai")

print("Setup complete!")


✓ Loaded .env from: /Users/carbonjo/Library/CloudStorage/Dropbox/AI/Agents-Langchain-llamaindex-MCP/MCP_Nov29-25/.env
Using Python: /Users/carbonjo/Library/CloudStorage/Dropbox/AI/Agents-Langchain-llamaindex-MCP/MCP_Nov29-25/venv/bin/python
✓ OPENAI_API_KEY loaded: sk-proj-Ih...12wA (length: 164)
✓ GEMINI_API_KEY loaded: AIzaSyDzWO...BRI4 (length: 39)
Setup complete!


## Helper Functions: MCP Server Connections

These functions help connect to our MCP servers.


In [2]:
# MCP Server configurations
# Note: Servers are automatically spawned when we connect
# IMPORTANT: Use PYTHON_EXECUTABLE to ensure servers use the venv Python with MCP installed
CODE_SERVER_PARAMS = StdioServerParameters(
    command=PYTHON_EXECUTABLE,  # Use venv Python, not system Python
    args=[str(Path("../servers/code_server.py").absolute())],
    env=None
)

DB_SERVER_PARAMS = StdioServerParameters(
    command=PYTHON_EXECUTABLE,  # Use venv Python, not system Python
    args=[str(Path("../servers/database_server.py").absolute())],
    env=None
)

DOC_SERVER_PARAMS = StdioServerParameters(
    command=PYTHON_EXECUTABLE,  # Use venv Python, not system Python
    args=[str(Path("../servers/document_server.py").absolute())],
    env=None
)

async def get_mcp_tools(server_params):
    """Get all available tools from an MCP server."""
    try:
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                tools_response = await session.list_tools()
                return {tool.name: tool for tool in tools_response.tools}
    except Exception as e:
        print(f"Error connecting to server: {e}")
        return {}

async def call_mcp_tool(server_params, tool_name, arguments):
    """Call a tool on an MCP server."""
    try:
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                result = await session.call_tool(tool_name, arguments)
                return result.content[0].text if result.content else "No output"
    except Exception as e:
        return f"Error calling tool: {str(e)}"

# Get all available tools from all servers
print("Loading MCP server tools...")
print("(This may take a moment as servers start up)\n")
all_tools = {}
for name, params in [("code", CODE_SERVER_PARAMS), ("database", DB_SERVER_PARAMS), ("document", DOC_SERVER_PARAMS)]:
    try:
        tools = await get_mcp_tools(params)
        all_tools.update({f"{name}_{k}": v for k, v in tools.items()})
        print(f"✓ {name} server: {len(tools)} tools")
    except Exception as e:
        print(f"✗ {name} server: Error - {e}")

if all_tools:
    print(f"\n✓ Total tools available: {len(all_tools)}")
else:
    print("\n⚠ Warning: No tools loaded. Check server paths and MCP installation.")
    print("Troubleshooting:")
    print("1. Make sure you're running from the notebooks/ directory")
    print("2. Check that server files exist in ../servers/")
    print("3. Verify MCP is installed: pip install mcp")


Loading MCP server tools...
(This may take a moment as servers start up)

✓ code server: 5 tools
✓ database server: 5 tools
✓ document server: 6 tools

✓ Total tools available: 16


## Method 1: Ollama Integration (Local Models)

Ollama allows you to run LLMs locally without API keys. First, make sure Ollama is installed and running.

### Setup Ollama

1. Install Ollama: https://ollama.ai
2. Pull a model: `ollama pull llama3` (or any other model)
3. Start Ollama service: `ollama serve`


In [3]:
def create_tools_description():
    """Create a description of all available MCP tools for the LLM."""
    description = "Available MCP Tools:\n\n"
    for tool_name, tool in all_tools.items():
        description += f"- {tool_name}: {tool.description}\n"
        if hasattr(tool, 'inputSchema') and 'properties' in tool.inputSchema:
            props = tool.inputSchema['properties']
            description += "  Parameters: " + ", ".join(props.keys()) + "\n"
        description += "\n"
    return description

def ollama_agent(prompt: str, model: str = "llama3", system_prompt: Optional[str] = None):
    """Use Ollama to process a prompt and decide on MCP tool usage."""
    if not OLLAMA_AVAILABLE:
        return "Ollama is not available. Please install it: pip install ollama"
    
    tools_desc = create_tools_description()
    
    if system_prompt is None:
        system_prompt = f"""You are an AI agent that can use MCP (Model Context Protocol) tools to help users.

{tools_desc}

When a user asks you to do something, analyze the request and determine:
1. Which tool(s) to use
2. What parameters to pass
3. How to interpret the results

Respond in JSON format with:
{{
    "reasoning": "Your thought process",
    "tool": "tool_name",
    "server": "code|database|document",
    "arguments": {{"param1": "value1", "param2": "value2"}},
    "response": "What to tell the user"
}}

If multiple tools are needed, provide a list of actions."""
    
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ]
        )
        return response['message']['content']
    except Exception as e:
        return f"Error calling Ollama: {str(e)}"

# Test Ollama connection
if OLLAMA_AVAILABLE:
    try:
        test_response = ollama.list()
        print("✓ Ollama is running!")
        # Ollama API returns a ListResponse object, not a dict
        # Access models as an attribute, and model name as m.model
        if hasattr(test_response, 'models'):
            models = [m.model for m in test_response.models]
            print(f"Available models: {models}")
        else:
            # Fallback for older API versions
            models = [m['name'] for m in test_response.get('models', [])]
            print(f"Available models: {models}")
    except Exception as e:
        print(f"⚠ Ollama connection error: {e}")
        print("Make sure Ollama is running: ollama serve")
        import traceback
        traceback.print_exc()
else:
    print("⚠ Ollama not installed")


✓ Ollama is running!
Available models: ['llama3.2-vision:latest', 'gemma3:4b', 'llama4:latest', 'llama3.2-vision:11b', 'llama3.2:latest', 'llama3.1:latest', 'mistral:latest', 'llama3:latest']


In [11]:
async def execute_ollama_agent_request(user_prompt: str, model: str = "gemma3:4b"):
    """Execute a user request using Ollama agent."""
    print(f"User request: {user_prompt}\n")
    
    # Get LLM response
    llm_response = ollama_agent(user_prompt, model=model)
    print(f"LLM Analysis:\n{llm_response}\n")
    
    # Try to parse JSON response
    try:
        # Extract JSON from response (might be wrapped in markdown)
        import re
        json_match = re.search(r'\{.*\}', llm_response, re.DOTALL)
        if json_match:
            decision = json.loads(json_match.group())
        else:
            print("Could not parse JSON from LLM response. Using direct interpretation.")
            decision = {"response": llm_response, "tool": None}
    except Exception as e:
        print(f"Error parsing LLM response: {e}")
        decision = {"response": llm_response, "tool": None}
    
    # Execute tool if specified
    # Handle both single tool and list of tools
    tool_data = decision.get("tool")
    server = decision.get("server")
    arguments = decision.get("arguments")
    
    # If tool is a list, take the first one (or handle multiple tools)
    if isinstance(tool_data, list):
        if len(tool_data) > 0:
            tool_name = tool_data[0]
            print(f"⚠ LLM returned multiple tools, using first: {tool_name}")
        else:
            return decision.get("response", "No action taken")
    elif tool_data:
        tool_name = tool_data
    else:
        return decision.get("response", "No action taken")
    
    # Validate we have all required fields
    if not tool_name or not server or not arguments:
        return decision.get("response", "No action taken - missing tool, server, or arguments")
    
    # Ensure tool_name is a string
    if not isinstance(tool_name, str):
        tool_name = str(tool_name)
    
    # Strip server prefix from tool name if present (e.g., "document_create_document" -> "create_document")
    # The LLM might return the prefixed name, but we need the actual tool name
    if isinstance(tool_name, str) and tool_name.startswith(f"{server}_"):
        actual_tool_name = tool_name[len(f"{server}_"):]
    else:
        actual_tool_name = tool_name
    
    # Map server name to params
    server_map = {
        "code": CODE_SERVER_PARAMS,
        "database": DB_SERVER_PARAMS,
        "document": DOC_SERVER_PARAMS
    }
    
    if server in server_map:
        print(f"Executing: {server}.{actual_tool_name} with arguments: {arguments}\n")
        try:
            result = await call_mcp_tool(server_map[server], actual_tool_name, arguments)
            print(f"Tool result:\n{result}\n")
            return result
        except Exception as e:
            error_msg = f"Error executing tool: {str(e)}"
            print(error_msg)
            return error_msg
    else:
        return f"Unknown server: {server}"

# Example: Use Ollama agent
if OLLAMA_AVAILABLE:
    # Uncomment to test:
    result = await execute_ollama_agent_request('''
    Create a document called 'test' with content the summary of 
    the book The Prince by Machiavelli''')#'Hello from Ollama agent!'")
    print(f"Final result: {result}")
    #print("Ready to use Ollama agent. Uncomment the example above to test.")
else:
    print("Ollama not available. Install with: pip install ollama")


User request: 
    Create a document called 'test' with content the summary of 
    the book The Prince by Machiavelli

LLM Analysis:
```json
{
    "reasoning": "The user wants to create a document. I need to use the document_create_document tool to achieve this. The content of the document should be a summary of \"The Prince\" by Machiavelli.",
    "tool": "document_create_document",
    "server": "document",
    "arguments": {
        "name": "test",
        "content": "Niccolò Machiavelli’s *The Prince* is a political treatise that outlines a pragmatic and often ruthless approach to acquiring and maintaining power. Written in the early 16th century, it argues that a ruler must prioritize the stability and security of the state above all else, even if this requires deception, manipulation, and the use of violence. Machiavelli distinguishes between virtue and vice, suggesting that a prince should be feared rather than loved if he cannot be both. The book emphasizes the importance of a

## Method 2: API-based Models (OpenAI & Gemini)

This method uses cloud-based LLM APIs. You'll need to provide API keys.


In [ ]:
# API Key Configuration
# API keys are read from environment variables or .env file
# The .env file is already loaded in cell 1 above
# Create a .env file in the project root with:
#   OPENAI_API_KEY=your-openai-key-here
#   GEMINI_API_KEY=your-gemini-key-here

# Note: .env file should already be loaded from cell 1
# This cell just reads the keys that were loaded

def get_api_key(provider: str) -> Optional[str]:
    """Get API key from environment variable or .env file."""
    env_key = os.getenv(f"{provider.upper()}_API_KEY")
    if env_key:
        # Mask the key for display (show only first/last few characters)
        if len(env_key) > 8:
            masked = env_key[:4] + "*" * (len(env_key) - 8) + env_key[-4:]
        else:
            masked = "*" * len(env_key)
        print(f"✓ Found {provider.upper()} API key: {masked}")
        return env_key
    else:
        print(f"⚠ {provider.upper()} API key not found")
        print(f"   Set it in .env file or environment variable: {provider.upper()}_API_KEY")
        if provider.lower() == "openai":
            print(f"   Get your key from: https://platform.openai.com/api-keys")
        elif provider.lower() == "gemini":
            print(f"   Get your key from: https://aistudio.google.com/app/apikey")
        return None

# Initialize API clients
openai_client = None
gemini_client = None
gemini_model_name = None  # Store which model worked

print("="*60)
print("🔑 API Key Setup")
print("="*60)
print("API keys are read from .env file or environment variables.")
print("\nTo set up API keys:")
print("  1. Create a .env file in the project root")
print("  2. Add your keys:")
print("     OPENAI_API_KEY=your-openai-key-here")
print("     GEMINI_API_KEY=your-gemini-key-here")
print("\n💡 Tip: You only need to set up the providers you want to use.")
print("   Ollama works without any keys (local models).")
print("="*60)
print()

# OpenAI Setup
if OPENAI_AVAILABLE:
    api_key = get_api_key("openai")
    if api_key:
        try:
            # Test the API key by making a simple call
            test_client = OpenAI(api_key=api_key)
            # Don't make an actual API call during setup, just create the client
            openai_client = test_client
            print("✓ OpenAI client initialized")
        except Exception as e:
            print(f"⚠ Error initializing OpenAI: {e}")
            print("   The API key might be invalid. Check your .env file.")
            openai_client = None
    else:
        print("⏭️  OpenAI skipped (you can set it up later if needed)")
else:
    print("⚠ OpenAI library not installed")

print()  # Add spacing

# Gemini Setup
if GEMINI_AVAILABLE:
    api_key = get_api_key("gemini")
    if api_key:
        genai.configure(api_key=api_key)
        gemini_client = None
        gemini_model_name = None
        
        # First, try to list available models to find the right one
        try:
            print("   Checking available Gemini models...")
            available_models = []
            model_names_with_prefix = []
            for m in genai.list_models():
                if 'generateContent' in m.supported_generation_methods:
                    full_name = m.name  # e.g., "models/gemini-2.5-flash"
                    short_name = full_name.replace('models/', '')  # e.g., "gemini-2.5-flash"
                    available_models.append(short_name)
                    model_names_with_prefix.append((full_name, short_name))
            
            if available_models:
                print(f"   Found {len(available_models)} available model(s)")
                # Try models in order of preference (newer models first)
                preferred_models = [
                    'gemini-2.5-flash',      # Latest flash
                    'gemini-2.5-pro',       # Latest pro
                    'gemini-2.0-flash',     # Previous version
                    'gemini-1.5-flash',     # Older but stable
                    'gemini-1.5-pro',       # Older but stable
                    'gemini-pro',           # Classic
                    'gemini-pro-latest'      # Latest alias
                ]
                
                # Try preferred models first
                for preferred in preferred_models:
                    if preferred in available_models:
                        try:
                            # Try with and without models/ prefix
                            for full_name, short_name in model_names_with_prefix:
                                if short_name == preferred:
                                    test_model = genai.GenerativeModel(full_name)
                                    gemini_client = test_model
                                    gemini_model_name = short_name  # Store without prefix for consistency
                                    print(f"✓ Gemini client initialized (using {short_name})")
                                    break
                            if gemini_client:
                                break
                        except Exception as e:
                            continue
                
                # If preferred models didn't work, try first available
                if gemini_client is None:
                    try:
                        first_full, first_short = model_names_with_prefix[0]
                        gemini_client = genai.GenerativeModel(first_full)
                        gemini_model_name = first_short
                        print(f"✓ Gemini client initialized (using first available: {first_short})")
                    except Exception as e:
                        print(f"⚠ Could not initialize with any model: {e}")
                        print(f"   Sample available models: {', '.join(available_models[:5])}")
                        gemini_client = None
            else:
                print("⚠ No models found with generateContent support")
                gemini_client = None
        except Exception as e:
            print(f"⚠ Error checking Gemini models: {e}")
            print("   Trying default model names...")
            # Fallback to trying common model names
            models_to_try = ['gemini-pro', 'gemini-1.5-pro', 'gemini-1.5-flash']
            for model_name in models_to_try:
                try:
                    test_model = genai.GenerativeModel(model_name)
                    gemini_client = test_model
                    gemini_model_name = model_name
                    print(f"✓ Gemini client initialized (using {model_name})")
                    break
                except:
                    continue
            
            if gemini_client is None:
                print("⚠ Could not initialize Gemini client")
                print("   Check your API key and available models at: https://ai.google.dev/gemini-api/docs/models/gemini")
    else:
        print("⏭️  Gemini skipped (you can set it up later if needed)")
        gemini_client = None
        gemini_model_name = None
else:
    print("⚠ Gemini library not installed")
    gemini_client = None
    gemini_model_name = None

print()
print("="*60)
print("✅ Setup Complete!")
print("="*60)
# Show summary
providers_available = []
if OLLAMA_AVAILABLE:
    providers_available.append("Ollama (local)")
if openai_client:
    providers_available.append("OpenAI")
if gemini_client:
    providers_available.append(f"Gemini ({gemini_model_name if gemini_model_name else 'default'})")

if providers_available:
    print(f"📋 Available providers: {', '.join(providers_available)}")
    print("\n💡 Use these providers in your agent calls:")
    if openai_client:
        print("   • provider='openai'")
    if gemini_client:
        print("   • provider='gemini'")
    if OLLAMA_AVAILABLE:
        print("   • provider='ollama'")
else:
    print("⚠ No providers available. Set up at least one above, or use Ollama (local, no key needed).")
print("="*60)


✓ Loaded .env file from: /Users/carbonjo/Library/CloudStorage/Dropbox/AI/Agents-Langchain-llamaindex-MCP/MCP_Nov29-25
🔑 API Key Setup
API keys are read from .env file or environment variables.

To set up API keys:
  1. Create a .env file in the project root
  2. Add your keys:
     OPENAI_API_KEY=your-openai-key-here
     GEMINI_API_KEY=your-gemini-key-here

💡 Tip: You only need to set up the providers you want to use.
   Ollama works without any keys (local models).

✓ Found OPENAI API key: sk-p************************************************************************************************************************************************************52gA
✓ OpenAI client initialized

✓ Found GEMINI API key: AIza*******************************BRI4
   Checking available Gemini models...
   Found 41 available model(s)
✓ Gemini client initialized (using gemini-2.5-flash)

✅ Setup Complete!
📋 Available providers: Ollama (local), OpenAI, Gemini (gemini-2.5-flash)

💡 Use these providers in your

In [ ]:
def openai_agent(prompt: str, model: str = "gpt-4o-mini", system_prompt: Optional[str] = None):
    """Use OpenAI to process a prompt and decide on MCP tool usage."""
    # If client not initialized, try to get key and create it
    if not openai_client:
        api_key = get_api_key("openai")
        if not api_key:
            return "OpenAI client not initialized. Please provide an API key in .env file."
        # Try to recreate the client
        try:
            from openai import OpenAI
            global openai_client
            openai_client = OpenAI(api_key=api_key)
        except Exception as e:
            return f"OpenAI client initialization failed: {e}"
    
    tools_desc = create_tools_description()
    
    if system_prompt is None:
        system_prompt = f"""You are an AI agent that can use MCP (Model Context Protocol) tools to help users.

{tools_desc}

When a user asks you to do something, analyze the request and determine:
1. Which tool(s) to use
2. What parameters to pass
3. How to interpret the results

Respond in JSON format with:
{{
    "reasoning": "Your thought process",
    "tool": "tool_name",
    "server": "code|database|document",
    "arguments": {{"param1": "value1", "param2": "value2"}},
    "response": "What to tell the user"
}}

If multiple tools are needed, provide a list of actions."""
    
    try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"} if "gpt-4" in model or "gpt-3.5" in model else None
        )
        return response.choices[0].message.content
    except Exception as e:
        error_msg = str(e)
        # Provide helpful error messages
        if "401" in error_msg or "invalid_api_key" in error_msg.lower():
            # Check what key is being used for debugging
            api_key_check = get_api_key("openai")
            if api_key_check:
                key_preview = f"{api_key_check[:10]}...{api_key_check[-4:]}"
                return f"❌ OpenAI API key is invalid or expired.\n   Current key: {key_preview}\n   Please check your .env file and get a new key from: https://platform.openai.com/api-keys\n   Note: Even if the key loads successfully, it may be invalid or expired."
            else:
                return f"❌ OpenAI API key not found. Please add OPENAI_API_KEY to your .env file.\n   Get a key from: https://platform.openai.com/api-keys"
        elif "429" in error_msg or "rate_limit" in error_msg.lower():
            return f"❌ OpenAI rate limit exceeded. Please wait a moment and try again."
        else:
            return f"❌ Error calling OpenAI: {error_msg}"

def gemini_agent(prompt: str, model: str = None, system_prompt: Optional[str] = None):
    """Use Google Gemini to process a prompt and decide on MCP tool usage."""
    if not gemini_client:
        return "Gemini client not initialized. Please provide an API key."
    
    # Use the model that worked during initialization, or the provided model
    if model is None:
        model = gemini_model_name if gemini_model_name else "gemini-pro"
    
    tools_desc = create_tools_description()
    
    if system_prompt is None:
        system_prompt = f"""You are an AI agent that can use MCP (Model Context Protocol) tools to help users.

{tools_desc}

When a user asks you to do something, analyze the request and determine:
1. Which tool(s) to use
2. What parameters to pass
3. How to interpret the results

Respond in JSON format with:
{{
    "reasoning": "Your thought process",
    "tool": "tool_name",
    "server": "code|database|document",
    "arguments": {{"param1": "value1", "param2": "value2"}},
    "response": "What to tell the user"
}}

If multiple tools are needed, provide a list of actions."""
    
    try:
        # Create model instance with the specified model name
        # Use the model that worked during initialization, or try the specified one
        if model and model != gemini_model_name:
            try:
                # Try with models/ prefix first, then without
                try:
                    model_client = genai.GenerativeModel(f"models/{model}")
                except:
                    model_client = genai.GenerativeModel(model)
            except:
                # If specified model fails, fall back to the working model
                model_client = gemini_client
                model = gemini_model_name
        else:
            model_client = gemini_client
            model = gemini_model_name if gemini_model_name else "gemini-pro"
            
        if not model_client:
            return "❌ Gemini client not initialized. Please set up Gemini API key in .env file."
            
        full_prompt = f"{system_prompt}\n\nUser request: {prompt}\n\nRespond in JSON format:"
        response = model_client.generate_content(full_prompt)
        return response.text
    except Exception as e:
        error_msg = str(e)
        # Provide helpful error messages
        if "404" in error_msg and "not found" in error_msg.lower():
            return f"❌ Gemini model '{model}' not found. Available models may have changed.\n   Re-run the setup cell to auto-detect available models.\n   Or check: https://ai.google.dev/gemini-api/docs/models/gemini"
        elif "401" in error_msg or "invalid" in error_msg.lower() and "key" in error_msg.lower():
            return f"❌ Gemini API key is invalid. Please check your .env file:\n   GEMINI_API_KEY=your-valid-key-here\n   Get a new key from: https://aistudio.google.com/app/apikey"
        else:
            return f"❌ Error calling Gemini: {error_msg}"


In [15]:
async def execute_api_agent_request(user_prompt: str, provider: str = "openai", model: str = None):
    """Execute a user request using API-based agent (OpenAI or Gemini)."""
    print(f"User request: {user_prompt}\n")
    
    # Select provider and model
    if provider.lower() == "openai":
        if not openai_client:
            return "❌ OpenAI client not initialized. Please provide an OpenAI API key in the setup cell above."
        if model is None:
            model = "gpt-4o-mini"
        llm_response = openai_agent(user_prompt, model=model)
    elif provider.lower() == "gemini":
        if not gemini_client:
            return "❌ Gemini client not initialized. Please provide a Gemini API key in the setup cell above."
        if model is None:
            # Use the model that worked during initialization
            model = gemini_model_name if gemini_model_name else "gemini-pro"
        llm_response = gemini_agent(user_prompt, model=model)
    else:
        return f"❌ Unknown provider: {provider}. Use 'openai' or 'gemini'."
    
    print(f"LLM Analysis ({provider}):\n{llm_response}\n")
    
    # Try to parse JSON response
    try:
        import re
        json_match = re.search(r'\{.*\}', llm_response, re.DOTALL)
        if json_match:
            decision = json.loads(json_match.group())
        else:
            # Try direct JSON parse
            decision = json.loads(llm_response)
    except Exception as e:
        print(f"Could not parse JSON from LLM response: {e}")
        print("Using direct interpretation.")
        decision = {"response": llm_response, "tool": None}
    
    # Execute tool if specified
    # Handle both single tool and list of tools
    tool_data = decision.get("tool")
    server = decision.get("server")
    arguments = decision.get("arguments")
    
    # If tool is a list, take the first one (or handle multiple tools)
    if isinstance(tool_data, list):
        if len(tool_data) > 0:
            tool_name = tool_data[0]
            print(f"⚠ LLM returned multiple tools, using first: {tool_name}")
        else:
            return decision.get("response", "No action taken")
    elif tool_data:
        tool_name = tool_data
    else:
        return decision.get("response", "No action taken")
    
    # Validate we have all required fields
    if not tool_name or not server or not arguments:
        return decision.get("response", "No action taken - missing tool, server, or arguments")
    
    # Ensure tool_name is a string
    if not isinstance(tool_name, str):
        tool_name = str(tool_name)
    
    # Strip server prefix from tool name if present (e.g., "document_create_document" -> "create_document")
    # The LLM might return the prefixed name, but we need the actual tool name
    if isinstance(tool_name, str) and tool_name.startswith(f"{server}_"):
        actual_tool_name = tool_name[len(f"{server}_"):]
    else:
        actual_tool_name = tool_name
    
    # Map server name to params
    server_map = {
        "code": CODE_SERVER_PARAMS,
        "database": DB_SERVER_PARAMS,
        "document": DOC_SERVER_PARAMS
    }
    
    if server in server_map:
        print(f"Executing: {server}.{actual_tool_name} with arguments: {arguments}\n")
        try:
            result = await call_mcp_tool(server_map[server], actual_tool_name, arguments)
            print(f"Tool result:\n{result}\n")
            return result
        except Exception as e:
            error_msg = f"Error executing tool: {str(e)}"
            print(error_msg)
            return error_msg
    else:
        return f"Unknown server: {server}"

# Example usage (uncomment to test):
if openai_client:
    result = await execute_api_agent_request("List all users in the database", provider="openai")
    print(f"Final result: {result}")

if gemini_client:
    result = await execute_api_agent_request("Create a Python script that calculates fibonacci numbers", provider="gemini")
    print(f"Final result: {result}")


User request: List all users in the database

LLM Analysis (openai):
❌ OpenAI API key is invalid or expired. Please check your .env file:
   OPENAI_API_KEY=your-valid-key-here
   Get a new key from: https://platform.openai.com/api-keys

Could not parse JSON from LLM response: Expecting value: line 1 column 1 (char 0)
Using direct interpretation.
Final result: ❌ OpenAI API key is invalid or expired. Please check your .env file:
   OPENAI_API_KEY=your-valid-key-here
   Get a new key from: https://platform.openai.com/api-keys
User request: Create a Python script that calculates fibonacci numbers

LLM Analysis (gemini):
```json
{
  "reasoning": "The user wants a Python script to calculate Fibonacci numbers. I will define a Python function for Fibonacci calculation and then use `code_write_file` to save it to a file named `fibonacci.py`.",
  "tool": "code_write_file",
  "server": "code",
  "arguments": {
    "file_path": "fibonacci.py",
    "content": "def fibonacci(n):\n    if n <= 0:\n   

## Interactive Agent Function

A unified function that lets you choose which LLM provider to use.


In [ ]:
async def intelligent_agent(user_prompt: str, provider: str = "ollama", model: str = None):
    """
    Intelligent agent that uses LLM to understand requests and execute MCP tools.
    
    Args:
        user_prompt: Natural language request from user
        provider: "ollama", "openai", or "gemini"
        model: Model name (optional, uses defaults if not provided)
    
    Returns:
        Result of the agent's actions
    """
    print(f"🤖 Using {provider.upper()} agent\n")
    print("=" * 60)
    
    if provider.lower() == "ollama":
        if not OLLAMA_AVAILABLE:
            return "Ollama is not available. Install with: pip install ollama"
        if model is None:
            model = "llama3"
        return await execute_ollama_agent_request(user_prompt, model=model)
    elif provider.lower() in ["openai", "gemini"]:
        if model is None:
            model = "gpt-4o-mini" if provider.lower() == "openai" else "gemini-pro"
        return await execute_api_agent_request(user_prompt, provider=provider, model=model)
    else:
        return f"Unknown provider: {provider}. Use 'ollama', 'openai', or 'gemini'"

# Example usage:
print("Ready to use intelligent agent!")
print("\n📋 Available providers:")
if OLLAMA_AVAILABLE:
    print("  ✓ Ollama (local)")
if openai_client:
    print("  ✓ OpenAI")
if gemini_client:
    print(f"  ✓ Gemini (using {gemini_model_name if gemini_model_name else 'default'})")
if not any([OLLAMA_AVAILABLE, openai_client, gemini_client]):
    print("  ⚠ No providers available - set up at least one in the cells above")

print("\n💡 Example commands:")
print("  - 'List all users in the database'")
print("  - 'Create a document called notes.txt with some content'")
print("  - 'Write a Python function to calculate factorial'")
print("  - 'Search for documents containing the word MCP'")
print("\n📝 Usage:")
print("  result = await intelligent_agent('your request', provider='ollama'|'openai'|'gemini')")
print("\n⚠ Important: Use the provider you set up above!")
print("  - If you set up Gemini, use provider='gemini'")
print("  - If you set up OpenAI, use provider='openai'")
print("  - If you have Ollama, use provider='ollama'")
print("\nUncomment below to test:")
# result = await intelligent_agent("List all database tables", provider="gemini")  # Use the provider you set up!
# print(f"\nFinal Result:\n{result}")


## Example: Multi-Step Workflow

Demonstrates how the agent can handle complex, multi-step requests.


In [ ]:
async def multi_step_workflow_example(provider: str = "ollama"):
    """Example of a complex multi-step workflow using the intelligent agent."""
    
    print("=" * 60)
    print("Multi-Step Workflow Example")
    print("=" * 60)
    print()
    
    steps = [
        "Query the database to get the count of users",
        "Create a Python script that prints 'Hello from MCP Agent'",
        "Create a document summarizing the database query results"
    ]
    
    for i, step in enumerate(steps, 1):
        print(f"Step {i}: {step}")
        result = await intelligent_agent(step, provider=provider)
        print(f"Result: {result}\n")
        print("-" * 60)
        print()

# Uncomment to run:
# await multi_step_workflow_example(provider="ollama")


## Summary

This notebook demonstrates:

1. **Ollama Integration**: Run LLMs locally without API keys
2. **OpenAI Integration**: Use GPT models with API keys
3. **Gemini Integration**: Use Google's Gemini models with API keys
4. **MCP Tool Integration**: All LLMs can use MCP servers to perform actions
5. **Intelligent Agent**: Unified interface for all providers

### Next Steps

- Try different models and providers
- Extend the agent to handle multi-turn conversations
- Add more sophisticated tool selection logic
- Implement error handling and retry logic
- Add conversation memory/history
